In [182]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import requests
import ast

In [183]:
V_T_df=pd.read_csv('data/spark_V.csv')
id_map=pd.read_csv('data/id_mapping.csv')
avg_show=pd.read_csv('data/avg_show_R.csv')
num_views=pd.read_csv('data/num_views.csv')

V_T_df['id'] = id_map['col1']
print len(V_T_df['id'].unique())
print id_map.head()

9856
   col1
0     1
1     5
2     6
3     7
4     8


In [184]:
series_df=pd.read_csv('data/series_data.csv')
series_df['real_genres']=series_df['genres'].apply(lambda x: ast.literal_eval(x))

series_df['in'] = series_df['id'].apply(lambda x: x in np.array(V_T_df['id']))
series_df=series_df[series_df['in']]
series_df=series_df.sort_values('id')
print len(series_df)


9751


In [185]:
V_T_df['in'] = V_T_df['id'].apply(lambda x: x in np.array(series_df['id']))
V_T_df = V_T_df[V_T_df['in']]
print len(V_T_df)

9751


In [186]:
avg_show['in']=avg_show['anime_id'].apply(lambda x: x in np.array(series_df['id']))
avg_show=avg_show[avg_show['in']]

num_views['in']=num_views['anime_id'].apply(lambda x: x in np.array(series_df['id']))
num_views=num_views[num_views['in']]

print len(num_views)
num_views.head()

9751


,anime_id,num_views,in
0,1,10175,True
1,5,4530,True
2,6,5824,True
3,7,1134,True
4,8,196,True


In [187]:
series_df['average_rating']=np.array(avg_show['avg_for_show'])
series_df['num_views']=np.array(num_views['num_views'])
item_mapping = np.array(series_df['id'])

In [188]:
url='https://anilist.co/api/'
cid='selzla-6acux'
sec='eGi4fmsY9pV64E1fSTWJJ1'
username='selzla'
params={'grant_type':"client_credentials",'client_id':cid,'client_secret':sec}
access=requests.post(url+'auth/access_token',data=params)
access_token=access.json()['access_token']

In [189]:
user_anime=requests.get(url+'user/'+username+'/animelist?access_token='+access_token)

In [190]:
completed = user_anime.json()['lists']['completed']

if 'watching' in user_anime.json()['lists']:
    watching = user_anime.json()['lists']['watching']
else:
    watching = []

if 'dropped' in user_anime.json()['lists']:
    dropped = user_anime.json()['lists']['dropped']
else:
    dropped = []

if 'on_hold' in user_anime.json()['lists']:
    on_hold = user_anime.json()['lists']['on_hold']
else:
    on_hold = []

if 'plan_to_watch' in user_anime.json()['lists']:
    plan_to_watch = user_anime.json()['lists']['plan_to_watch']
else:
    plan_to_watch = []

In [191]:
scores = []
for i in range(len(completed)):
    title=completed[i]['anime']['title_english'].encode("utf-8")
    anime_id=completed[i]['anime']['id']
    score=completed[i]['score_raw']
    scores.append([anime_id, score])

In [192]:
def isAdult(ID):
    if len(list(series_df[series_df['id']==ID]['adult'])):
        return list(series_df[series_df['id']==ID]['adult'])[0]
    else:
        return None

In [193]:
def hasGenre(ID, genre):
    if len(list(series_df[series_df['id']==ID]['real_genres']))>0:
        return genre in list(series_df[series_df['id']==ID]['real_genres'])[0]
    else:
        return None

In [194]:
def StartYear(ID):
    if len(list(series_df[series_df['id']==ID]['start_date']))>0:
        if type(list(series_df[series_df['id']==ID]['start_date'])[0])==str:
            return int(list(series_df[series_df['id']==ID]['start_date'])[0][:4])
    else:
        return None

In [195]:
def numEpisodes(ID):
    if len(list(series_df[series_df['id']==ID]['total_episodes']))>0:
        return list(series_df[series_df['id']==ID]['total_episodes'])[0]
    else: 
        return None

In [196]:
def Type(ID):
    if len(list(series_df[series_df['id']==ID]['type']))>0:
        return list(series_df[series_df['id']==ID]['type'])[0]
    else:
        return None

In [197]:
titles={}
for i in range(len(series_df)):
    ID = np.array(series_df['id'])[i]
    title = np.array(series_df['title_english'])[i]
    titles[ID] = title


In [198]:
series_df.head()

,adult,airing_status,description,genres,id,season,start_date,title_english,total_episodes,type,real_genres,in,average_rating,num_views
6442,False,finished airing,"Enter a world in the distant future, where Bou...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",1,NaN,1998-04-03T00:00:00+09:00,Cowboy Bebop,26,TV,"[Action, Adventure, Comedy, Drama, Sci-Fi]",True,85.094644,10175
8796,False,finished airing,"As the Cowboy Bebop crew travels the stars, th...","['Action', 'Drama', 'Mystery', 'Sci-Fi']",5,NaN,2001-09-01T00:00:00+09:00,Cowboy Bebop: Knockin' on Heaven's Door,1,Movie,"[Action, Drama, Mystery, Sci-Fi]",True,80.694702,4530
9958,False,finished airing,Trigun takes place in the distant future on a ...,"['Action', 'Comedy', 'Sci-Fi']",6,NaN,1998-04-01T00:00:00+09:00,Trigun,26,TV,"[Action, Comedy, Sci-Fi]",True,79.888049,5824
6528,False,finished airing,Witches are individuals with special powers li...,"['Action', 'Drama', 'Mystery', 'Supernatural']",7,NaN,2002-07-02T00:00:00+09:00,Witch Hunter Robin,26,TV,"[Action, Drama, Mystery, Supernatural]",True,71.701940,1134
2983,False,finished airing,It is the dark century and the people are suff...,"['Adventure', 'Fantasy', 'Supernatural']",8,NaN,2004-09-30T00:00:00+09:00,Beet the Vandel Buster,52,TV,"[Adventure, Fantasy, Supernatural]",True,70.015306,196


In [199]:
popularity={}
for i in range(len(series_df)):
    ID = np.array(series_df['id'])[i]
    avr = np.array(series_df['num_views'])[i]
    popularity[ID] = avr

In [200]:
user=np.array([row for row in scores if row[0] in item_mapping])

#user=np.array([[row[0]-1, row[1]] for row in user])
# print user

In [201]:
V=np.array(V_T_df[['col'+str(i) for i in range(1,11)]]).T
print V.shape[0], V.shape[1]

10 9751


In [202]:
avg_show.head()

,Unnamed: 0,anime_id,avg_for_show,in
0,0,1,85.094644,True
1,1,5,80.694702,True
2,2,6,79.888049,True
3,3,7,71.701940,True
4,4,8,70.015306,True


In [203]:
average_ratings={}
for i in range(len(avg_show)):
    ID = np.array(series_df['id'])[i]
    avr = np.array(series_df['average_rating'])[i]
    average_ratings[ID] = avr
print average_ratings[1]

85.0946437346


In [204]:
user_vector = np.zeros(V.shape[1])
for row in user:
    user_vector[list(item_mapping).index(row[0])]=row[1]+74-np.mean(user[:,1])-average_ratings[row[0]]

In [205]:
new_user=user_vector[np.where(user_vector!=0)]
new_V=V.T[np.where(user_vector!=0)].T
s=np.linalg.lstsq(new_V.T,new_user.T)[0].T
new_R=np.dot(s,V)
for i in range(len(new_R)):
    ratio = (popularity[item_mapping[i]]-np.sqrt(popularity[item_mapping[i]]))/popularity[item_mapping[i]]
    new_R[i]=(new_R[i]+average_ratings[item_mapping[i]]+np.mean(user[:,1])-74)*ratio

In [206]:
print type(new_R)

<type 'numpy.ndarray'>


In [207]:
seen = [row[0] for row in scores]
others = [watching, dropped, on_hold, plan_to_watch]
for x in others:
    for i in range(len(x)):
        seen.append(x[i]['anime']['id'])

In [208]:
series_df['predicted_ratings'] = new_R

In [209]:
def startyear(string):
    try:
        result = int(string[:4])
    except:
        result = 1
    return 1

In [210]:
f_type = (True, 'TV')
f_genres = (False, 'Adventure')
f_year = (False, 2014, 2016)
f_episodes = (False, 1, 100)
f_adult = (False, True)
f_popularity = (False, 5000)

if f_type[0]:
    series_df = series_df[series_df['type']==f_type[1]]
if f_genres[0]:
    series_df['has_genre'] = series_df['real_genres'].apply(lambda x: f_genres[1] in x)
    series_df = series_df[series_df['has_genre']]
if f_year[0]:
    series_df['start_year'] = series_df['start_date'].apply(startyear)
    series_df = series_df[series_df['start_year']>=f_year[1]]
    series_df = series_df[series_df['start_year']<=f_year[2]]
if f_episodes[0]:
    series_df = series_df[series_df['total_episodes']>=f_episodes[1]]
    series_df = series_df[series_df['total_episodes']<=f_episodes[2]]
if f_adult[0]:
    series_df = series_df[series_df['adult']==f_adult[1]]
if f_popularity[0]:
    series_df = series_df[series_df['num_views']>=f_popularity[1]]
series_df = series_df.sort_values('predicted_ratings', ascending=False)
series_df['not_seen'] = series_df['id'].apply(lambda x: x not in seen)
series_df = series_df[series_df['not_seen']]
print series_df[['title_english', 'predicted_ratings']].head(15)


    
    

                        title_english  predicted_ratings
9514              LUPIN THE 3rd PART4         114.558246
8178    Pretty Rhythm: Dear My Future         109.220913
1062   Detective Opera Milky Holmes 2         108.316188
1933                    Sailor Moon S         105.204882
10128             Yumemiru Toppo Jijo         104.627241
6222                      Tetsujin 28         104.433200
3619         Sailor Moon Sailor Stars         102.320681
6649                      Getter Robo         101.817479
1632           Fist of the North Star         101.292929
3660                The Tatami Galaxy         100.984817
70        Showa Genroku Rakugo Shinju         100.889736
2273                      Hyouge Mono         100.646061
5213          Neon Genesis Evangelion         100.374883
2349                 The Flying House         100.268210
7705         Revolutionary Girl Utena         100.109670


In [211]:
G = []
for x in series_df['real_genres']:
    for y in x:
        if y not in G:
            G.append(y)
print G

['Action', 'Adventure', 'Comedy', 'Music', 'Sports', 'Mystery', 'Drama', 'Romance', 'Mahou Shoujo', 'Mecha', 'Sci-Fi', 'Fantasy', 'Psychological', 'Ecchi', 'Supernatural', 'Horror', 'Thriller', 'Slice of Life', '']


In [212]:
np.argsort(-1*new_R)

array([6551, 7399, 8522, ..., 5302, 8036, 3571])

In [213]:
np.sqrt(9)


3.0